<a href="https://colab.research.google.com/github/phylars/ILEE_CSK/blob/ipynb/2D_mode_calculating_effective_area.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preparation**

Please transfer you confocal format files to raw (stack) tiff images using ImageJ/FIJI, and put them in a new folder.
*   If you wish to run the pipeline in Google Colab, please upload the tiff folder to your Google Drive


In [ ]:
#@title Mount to your google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Install ILEE_CSK and dependencies
!pip install -U ILEE_CSK
import ILEE_CSK
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scikit_posthocs as sp

In [ ]:
#@title Inspect the data structure of your tif image
#@markdown All your image samples should be taken under the same confocal setting. Please input the file path of any one of your image file in this batch of experiment, below. It can be local (for jupyter notebook) or in your Google Drive (Google Colab):

example_tif_file_path = ""  #@param {type: "string"}
img_example = io.imread(example_tif_file_path)
print('The image array structure is', img_example.shape)

#@markdown (Optional/Recommended): This pipeline also provide data visualization and significance analysis. To enable this, please name your tiff files as (group_name)-(repeat_index).tif, such as mock-1.tif, mock-2.tif, or treatment1-1.tif ... The hyphen '-' must occur only once.

The image array structure needs to meet 2 requirements:

*   Your image array structure should be 3D or 4D (seeing 3 or 4 numbers in the tuple). If it is 2D, you may be using a projected image, ILEE_CSK fully supports it but it cannot be processed in this pipeline. If it has 5 or more dimension, this is an abnormal data structure and please submit a bug report in the Github.
*   If it is 4D, the dimension with lowest size should be your channel, and the the dimension with second lowest size should be your z-axis.

To learn which channel is your objective, please run the following step to inspect your example image:

In [ ]:
#@title Inspect the channels of the example image
def align_image_dimension(img):
    img_size = np.array(img.shape)
    dimension = img_size.shape[0]
    df_dimension = pd.DataFrame(data = img_size, columns = ['dimension_size'])
    df_dimension['dimension_index'] = range(img_size.shape[0])
    df_dimension = df_dimension.sort_values(by = ['dimension_size', 'dimension_index'])
    if (dimension <= 2):
      print ('Error: image have less than 3 dimensions; please check data structure')
      return()
    elif (dimension == 3):
      result = np.moveaxis(img, source = np.array(df_dimension['dimension_index']), destination = np.array([0,1,2]))
    elif (dimension == 4):
      result = np.moveaxis(img, source = np.array(df_dimension['dimension_index']), destination = np.array([0,1,2,3]))
    else:
      print('image has more than 4 dimension (x, y, z, and channel); please check data structure')
      return()
    return (result)

if(len(img_example.shape) == 3):
  print('Your image sample is a 3D array (does not have channel dimension). Please set objective_channel to None in the next step.')
  print('\n')
  img_example = align_image_dimension(img_example)
  img_example_p = np.amax(img_example, axis=0)
  print('Below is a z-axis projected of your image sample:')
  plt.figure(figsize = (5, 5))
  plt.imshow(img_example_p)
  plt.axis('off')
  plt.show()
elif(len(img_example.shape) == 4):
  print('Your image sample is a 4D array (contains channel dimension). The projection image of each channel is shown below:')
  print('\n')
  img_example = align_image_dimension(img_example)
  for i in range(img_example.shape[0]):
    img_example_p = np.amax(img_example[i,:,:,:], axis=0)
    print('channel', i)
    plt.figure(figsize = (5, 5))
    plt.imshow(img_example_p)
    plt.axis('off')
    plt.show()
    print('\n')
print('Please set the objective_channel to the correct channel index for cytoskeleton in the next step.')

In [ ]:
#@title Input the image folder containing all the raw tiffs in this experiment

folder_path = ''  #@param {type: "string"}

In [ ]:
#@title Test total cell segmentation arguments and output effective area data
#@markdown The defualt setting is provided as below. Please test different arguments until you find satisfied results:

#@markdown global_thres_fold determines how high the is global threshold is set. Default setting is 4.5.

#@markdown hole_thres determines filling holes that is lower than a give area by pixel. The 'default' string is 1/256 of the total image. Please make sure you include the quotation mark if you are using 'default'.

#@markdown global_thres_fold determines deleting small object that is lower than a given area. Default setting is 3.

from ILEE_CSK import test_and_call_areas_of_folder
objective_channel_index = 0  #@param {type: "number"}
global_thres_fold = 4.5 #@param {type: "number"}
hole_thres = 'default' #@param
objective_thres = 3 #@param {type: "number"}
df_areas = test_and_call_areas_of_folder (folder_path = folder_path,
                                          obj_channel = objective_channel_index,
                                          global_thres_fold = global_thres_fold,
                                          hole_thres = hole_thres,
                                          objective_thres = objective_thres)


In [ ]:
#@title Output your area data
#@markdown The data will be output as excel. You can merge results of different outputs as long as the excel structure is maintained.

areas_file_name = "your_result.xlsx" #@param {type: "string"}
output_path = folder_path + '/' + areas_file_name
df_areas.to_excel(output_path)

#@markdown The excel is saved in your tiff image folder.